In [246]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import tensorflow as tf

In [247]:
WINDOW_SIZE = 10
WINDOW_STEP_SIZE = 1
EPOCHS = 2
STEPS_PER_EPOCH = 1000

In [248]:
ROOTDIR_DATASET = './KukaVelocityDataset'
kuka_column_names_path = os.path.join(ROOTDIR_DATASET, 'KukaColumnNames.npy')
kuka_column_names = np.load(kuka_column_names_path)
kuka_column_names.reshape((1, -1))

array([['action', 'machine_nameKuka Robot_apparent_power',
        'machine_nameKuka Robot_current',
        'machine_nameKuka Robot_frequency',
        'machine_nameKuka Robot_phase_angle',
        'machine_nameKuka Robot_power',
        'machine_nameKuka Robot_power_factor',
        'machine_nameKuka Robot_reactive_power',
        'machine_nameKuka Robot_voltage', 'sensor_id1_AccX',
        'sensor_id1_AccY', 'sensor_id1_AccZ', 'sensor_id1_GyroX',
        'sensor_id1_GyroY', 'sensor_id1_GyroZ', 'sensor_id1_q1',
        'sensor_id1_q2', 'sensor_id1_q3', 'sensor_id1_q4',
        'sensor_id1_temp', 'sensor_id2_AccX', 'sensor_id2_AccY',
        'sensor_id2_AccZ', 'sensor_id2_GyroX', 'sensor_id2_GyroY',
        'sensor_id2_GyroZ', 'sensor_id2_q1', 'sensor_id2_q2',
        'sensor_id2_q3', 'sensor_id2_q4', 'sensor_id2_temp',
        'sensor_id3_AccX', 'sensor_id3_AccY', 'sensor_id3_AccZ',
        'sensor_id3_GyroX', 'sensor_id3_GyroY', 'sensor_id3_GyroZ',
        'sensor_id3_q1', 'sensor_i

In [249]:
kuka_normal_path = os.path.join(ROOTDIR_DATASET, 'KukaNormal.npy')
kuka_normal = np.load(kuka_normal_path)
kuka_normal = np.hstack((kuka_normal, np.zeros(kuka_normal.shape[0]).reshape(-1, 1) ))
kuka_normal = kuka_normal.astype('float32')

kuka_normal = np.vstack((kuka_column_names, kuka_normal))
df_kuka_normal = pd.DataFrame(kuka_normal)
df_kuka_normal.columns = df_kuka_normal.iloc[0]
df_kuka_normal = df_kuka_normal.iloc[1:]

df_kuka_normal.drop("anomaly", axis=1, inplace=True)
df_kuka_normal = df_kuka_normal.reset_index(drop=True)
# df_kuka_normal['machine_nameKuka Robot_time'] = df_kuka_normal.index.astype(float) /  df_kuka_normal['machine_nameKuka Robot_frequency'].astype(float)

df_kuka_normal

,action,machine_nameKuka Robot_apparent_power,machine_nameKuka Robot_current,machine_nameKuka Robot_frequency,machine_nameKuka Robot_phase_angle,machine_nameKuka Robot_power,machine_nameKuka Robot_power_factor,machine_nameKuka Robot_reactive_power,machine_nameKuka Robot_voltage,sensor_id1_AccX,...,sensor_id7_AccY,sensor_id7_AccZ,sensor_id7_GyroX,sensor_id7_GyroY,sensor_id7_GyroZ,sensor_id7_q1,sensor_id7_q2,sensor_id7_q3,sensor_id7_q4,sensor_id7_temp
0,0.0,163.1622,0.976771,50.0,320.7093,126.283424,0.773975,-103.30755,228.32259,-0.6899414,...,-0.00390625,1.0009766,0.0,0.0,0.0,-0.08373314,0.15915737,-0.8742613,0.4509156,180.24
1,0.0,163.1622,0.976771,50.0,320.7093,126.283424,0.773975,-103.30755,228.32259,-0.6894531,...,-0.0078125,1.0014648,0.0,0.0,0.0,-0.08373314,0.15915737,-0.8742613,0.4509156,180.24
2,0.0,163.1622,0.976771,50.0,320.7093,126.283424,0.773975,-103.30755,228.32259,-0.68896484,...,-0.0034179688,1.0004883,0.0,0.0,0.0,-0.08373314,0.15915737,-0.8742613,0.4509156,180.24
3,0.0,163.1622,0.976771,50.0,320.7093,126.283424,0.773975,-103.30755,228.32259,-0.6899414,...,-0.0048828125,1.0014648,0.0,0.0,0.0,-0.08373314,0.15915737,-0.8742613,0.4509156,180.24
4,0.0,163.1622,0.976771,50.0,320.7093,126.283424,0.773975,-103.30755,228.32259,-0.6875,...,-0.005859375,0.99902344,0.0,0.0,0.0,-0.08373314,0.15915737,-0.8742613,0.4509156,180.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233787,0.0,160.46246,0.993556,50.0,321.58087,125.72502,0.783517,-99.69756,231.03653,-0.75439453,...,-0.0024414062,1.0004883,0.0,0.0,0.0,-0.17272379,-0.048043706,0.25176758,0.951037,180.24
233788,0.0,160.46246,0.993556,50.0,321.58087,125.72502,0.783517,-99.69756,231.03653,-0.75097656,...,-0.0029296875,1.0009766,0.0,0.0,0.0,-0.17272379,-0.048043706,0.25176758,0.951037,180.24
233789,0.0,160.46246,0.993556,50.0,321.58087,125.72502,0.783517,-99.69756,231.03653,-0.75439453,...,-0.0034179688,0.9995117,0.0,0.0,0.0,-0.17272379,-0.048043706,0.25176758,0.951037,180.24
233790,0.0,160.46246,0.993556,50.0,321.58087,125.72502,0.783517,-99.69756,231.03653,-0.7626953,...,-0.0048828125,1.0,0.0,0.061035156,0.0,-0.17293215,-0.048178624,0.2516482,0.95102394,180.24


In [250]:
df_kuka_normal['action'].astype(float).unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31.])

In [251]:
kuka_slow_path = os.path.join(ROOTDIR_DATASET, 'KukaSlow.npy')
kuka_slow = np.load(kuka_slow_path)
kuka_slow = kuka_slow.astype('float32')

kuka_slow = np.vstack((kuka_column_names, kuka_slow))

df_kuka_slow = pd.DataFrame(kuka_slow)
df_kuka_slow.columns = df_kuka_slow.iloc[0]
df_kuka_slow = df_kuka_slow.iloc[1:]
df_kuka_slow = df_kuka_slow.reset_index(drop=True)
df_kuka_slow.drop("anomaly", axis=1, inplace=True)
# df_kuka_slow['machine_nameKuka Robot_time'] = df_kuka_slow.index.astype(float) /  df_kuka_slow['machine_nameKuka Robot_frequency'].astype(float)
df_kuka_slow

,action,machine_nameKuka Robot_apparent_power,machine_nameKuka Robot_current,machine_nameKuka Robot_frequency,machine_nameKuka Robot_phase_angle,machine_nameKuka Robot_power,machine_nameKuka Robot_power_factor,machine_nameKuka Robot_reactive_power,machine_nameKuka Robot_voltage,sensor_id1_AccX,...,sensor_id7_AccY,sensor_id7_AccZ,sensor_id7_GyroX,sensor_id7_GyroY,sensor_id7_GyroZ,sensor_id7_q1,sensor_id7_q2,sensor_id7_q3,sensor_id7_q4,sensor_id7_temp
0,10.0,216.6106,1.201105,50.0,330.75912,189.01265,0.872592,-105.793724,228.44539,-0.6298828,...,0.092285156,1.0224609,-0.9765625,-9.033203,40.222168,-0.21068183,0.0138271935,-0.33694983,0.9175439,180.24
1,10.0,216.6106,1.201105,50.0,330.75912,189.01265,0.872592,-105.793724,228.44539,-0.6298828,...,0.0009765625,1.0126953,-0.18310547,-2.0751953,6.713867,-0.21132195,0.010657524,-0.31755194,0.9243323,180.24
2,10.0,216.6106,1.201105,49.951218,330.75912,189.01265,0.872592,-105.793724,228.44539,-0.49023438,...,0.056152344,1.0195312,0.0,-0.18310547,-0.12207031,-0.20851757,0.014488422,-0.31647766,0.92528504,180.24
3,10.0,216.6106,1.201105,49.951218,330.75912,189.01265,0.872592,-105.793724,228.44539,-0.62353516,...,-0.022460938,0.9892578,0.061035156,0.18310547,-0.30517578,-0.20604037,0.017369088,-0.31614384,0.9259043,180.24
4,10.0,216.6106,1.201105,49.951218,330.75912,189.01265,0.872592,-105.793724,228.44539,-0.5917969,...,-0.026855469,0.98779297,-0.12207031,0.12207031,0.6713867,-0.2039672,0.019725205,-0.3158316,0.92642254,180.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41533,0.0,216.71846,1.265179,49.951218,329.32294,186.39445,0.860077,-110.552055,228.7596,-0.5986328,...,0.0068359375,1.0117188,0.0,-9.82666,42.175293,-0.15644586,-0.108547136,0.3916207,0.90020853,180.24
41534,0.0,216.71846,1.265179,49.951218,329.32294,186.39445,0.860077,-110.552055,228.7596,-0.5986328,...,0.015136719,0.99121094,-0.12207031,-9.70459,41.137695,-0.15205419,-0.120020285,0.42289206,0.8852327,180.24
41535,0.0,216.71846,1.265179,49.951218,329.32294,186.39445,0.860077,-110.552055,228.7596,-0.5986328,...,-0.0073242188,1.0136719,0.0,-9.643555,41.992188,-0.14758018,-0.1305585,0.45207474,0.8699443,180.24
41536,0.0,216.71846,1.265179,49.951218,329.32294,186.39445,0.860077,-110.552055,228.7596,-0.5986328,...,0.020996094,0.9975586,-0.48828125,-9.94873,40.527344,-0.14264664,-0.14130676,0.48235497,0.8526535,180.24


In [252]:
a: np.ndarray = df_kuka_slow['action'].astype(float).unique()
a.sort()
a

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
       27., 28., 29., 30., 31.])

In [253]:
class Encoder(tf.keras.Model):
    def __init__(self, window_size):
        super(Encoder, self).__init__()
        # n_feature = 86
        # Input size: (window_size, n_feature)
        self.conv1d_1 = tf.keras.layers.Conv1D(filters=window_size*2, kernel_size=7, strides=1, padding='valid')
        # size: (window_size*2, 80)
        self.conv1d_2 = tf.keras.layers.Conv1D(filters=window_size*4, kernel_size=4, strides=2, padding='same')
        # size: (window_size*4, 40)
        self.conv1d_3 = tf.keras.layers.Conv1D(filters=window_size*8, kernel_size=4, strides=2, padding='same')
        # size: (window_size*8, 20)
        self.conv1d_4 = tf.keras.layers.Conv1D(filters=window_size*16, kernel_size=4, strides=2, padding='same')
        # size: (window_size*16, 10)
        self.conv1d_5 = tf.keras.layers.Conv1D(filters=window_size*32, kernel_size=10, strides=2, padding='same')
        # size: (window_size*32, 1)
        
        self.batch_norm_1 = tf.keras.layers.BatchNormalization()
        self.batch_norm_2 = tf.keras.layers.BatchNormalization()
        self.batch_norm_3 = tf.keras.layers.BatchNormalization()
        self.batch_norm_4 = tf.keras.layers.BatchNormalization()

    def build(self, input_shape):
        super(Encoder, self).build(input_shape)

    def call(self, X):
        print(X.shape)
        X = self.conv1d_1(X)
        X = self.batch_norm_1(X)
        X = tf.nn.leaky_relu(X)

        print(X.shape)
        X = self.conv1d_2(X)
        X = self.batch_norm_2(X)
        X = tf.nn.leaky_relu(X)

        print(X.shape)
        X = self.conv1d_3(X)
        X = self.batch_norm_3(X)
        X = tf.nn.leaky_relu(X)

        print(X.shape)
        X = self.conv1d_4(X)
        X = self.batch_norm_4(X)
        X = tf.nn.leaky_relu(X)
        
        print(X.shape)
        X = self.conv1d_5(X)
        print(X.shape)
        
        return X


class Decoder(tf.keras.Model):
    def __init__(self, window_size):
        super(Decoder, self).__init__()
        # Input size: (window_size*32, 1)
        self.conv_transpose_1d_1 = tf.keras.layers.Conv1DTranspose(filters=window_size*16, kernel_size=10, strides=2, padding='same')
        # size: (window_size*16, 10)
        self.conv_transpose_1d_2 = tf.keras.layers.Conv1DTranspose(filters=window_size*8, kernel_size=4, strides=2, padding='same')
        # size: (window_size*8, 20)
        self.conv_transpose_1d_3 = tf.keras.layers.Conv1DTranspose(filters=window_size*4, kernel_size=4, strides=2, padding='same')
        # size: (window_size*4, 40)
        self.conv_transpose_1d_4 = tf.keras.layers.Conv1DTranspose(filters=window_size*2, kernel_size=4, strides=2, padding='same')
        # size: (window_size*2, 80)
        self.conv_transpose_1d_5 = tf.keras.layers.Conv1DTranspose(filters=window_size, kernel_size=7, strides=1, padding='valid')
        # size: (window_size, 86)
        
        self.batch_norm_1 = tf.keras.layers.BatchNormalization()
        self.batch_norm_2 = tf.keras.layers.BatchNormalization()
        self.batch_norm_3 = tf.keras.layers.BatchNormalization()
        self.batch_norm_4 = tf.keras.layers.BatchNormalization()

    def build(self, input_shape):
        super(Decoder, self).build(input_shape)

    def call(self, X):
        print(X.shape)
        X = self.conv_transpose_1d_1(X)
        X = self.batch_norm_1(X)
        X = tf.nn.relu(X)
        
        print(X.shape)
        X = self.conv_transpose_1d_2(X)
        X = self.batch_norm_2(X)
        X = tf.nn.relu(X)

        print(X.shape)
        X = self.conv_transpose_1d_3(X)
        X = self.batch_norm_3(X)
        X = tf.nn.relu(X)
        
        print(X.shape)
        X = self.conv_transpose_1d_4(X)
        X = self.batch_norm_4(X)
        X = tf.nn.relu(X)
        
        print(X.shape)
        X = self.conv_transpose_1d_5(X)
        print(X.shape)
        
        return X
    

class AutoEncoder(tf.keras.Model):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = Encoder(WINDOW_SIZE)
        self.decoder = Decoder(WINDOW_SIZE)

    def build(self, input_shape):
        super(AutoEncoder, self).build(input_shape)

    def call(self, X):
        latent = self.encoder(X)
        X = self.decoder(latent)

        return X

In [254]:
df_kuka_normal.iloc[:1000]

,action,machine_nameKuka Robot_apparent_power,machine_nameKuka Robot_current,machine_nameKuka Robot_frequency,machine_nameKuka Robot_phase_angle,machine_nameKuka Robot_power,machine_nameKuka Robot_power_factor,machine_nameKuka Robot_reactive_power,machine_nameKuka Robot_voltage,sensor_id1_AccX,...,sensor_id7_AccY,sensor_id7_AccZ,sensor_id7_GyroX,sensor_id7_GyroY,sensor_id7_GyroZ,sensor_id7_q1,sensor_id7_q2,sensor_id7_q3,sensor_id7_q4,sensor_id7_temp
0,0.0,163.1622,0.976771,50.0,320.7093,126.283424,0.773975,-103.30755,228.32259,-0.6899414,...,-0.00390625,1.0009766,0.0,0.0,0.0,-0.08373314,0.15915737,-0.8742613,0.4509156,180.24
1,0.0,163.1622,0.976771,50.0,320.7093,126.283424,0.773975,-103.30755,228.32259,-0.6894531,...,-0.0078125,1.0014648,0.0,0.0,0.0,-0.08373314,0.15915737,-0.8742613,0.4509156,180.24
2,0.0,163.1622,0.976771,50.0,320.7093,126.283424,0.773975,-103.30755,228.32259,-0.68896484,...,-0.0034179688,1.0004883,0.0,0.0,0.0,-0.08373314,0.15915737,-0.8742613,0.4509156,180.24
3,0.0,163.1622,0.976771,50.0,320.7093,126.283424,0.773975,-103.30755,228.32259,-0.6899414,...,-0.0048828125,1.0014648,0.0,0.0,0.0,-0.08373314,0.15915737,-0.8742613,0.4509156,180.24
4,0.0,163.1622,0.976771,50.0,320.7093,126.283424,0.773975,-103.30755,228.32259,-0.6875,...,-0.005859375,0.99902344,0.0,0.0,0.0,-0.08373314,0.15915737,-0.8742613,0.4509156,180.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3.0,182.85611,0.888157,49.951218,331.71808,161.03116,0.880644,-86.62533,228.64078,-1.1547852,...,-0.015625,0.9663086,0.18310547,15.50293,-68.42041,-0.012764361,0.19107243,-0.9432588,0.27127686,180.24
996,3.0,182.85611,0.888157,49.951218,331.71808,161.03116,0.880644,-86.62533,228.64078,-1.0009766,...,0.020507812,1.0512695,0.79345703,16.235352,-66.04004,0.0069525233,0.19711855,-0.95612884,0.21659552,180.24
997,3.0,182.85611,0.888157,49.951218,331.71808,161.03116,0.880644,-86.62533,228.64078,-0.7607422,...,-0.020019531,1.078125,0.061035156,16.296387,-67.50488,0.026330445,0.20216514,-0.9656332,0.16120943,180.24
998,3.0,182.85611,0.888157,49.951218,331.71808,161.03116,0.880644,-86.62533,228.64078,-1.2446289,...,0.02734375,0.9794922,0.0,10.864258,-43.21289,0.042163517,0.20523992,-0.9713556,0.11210313,180.24


In [255]:
def sliding_window(data, window_size, step_size):
    windows = []
    for i in range(0, data.shape[0] - window_size + 1, step_size):
        windows.append(data.iloc[i:i + window_size].to_numpy().astype('float32'))
    
    x = np.array(windows).astype('float32')
    return np.swapaxes(x,0,1)

train_data = sliding_window(df_kuka_normal, WINDOW_SIZE, WINDOW_STEP_SIZE)

In [256]:
autoencoder: AutoEncoder = AutoEncoder()
autoencoder.build(train_data.shape)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

autoencoder.compile(optimizer=optimizer, loss='mse')
autoencoder.fit(train_data, train_data, batch_size=1024, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS, callbacks=[callback])
autoencoder.summary()


Epoch 1/2
(None, 116892, 86)
(None, 116892, 20)
(None, 58446, 40)
(None, 29223, 80)
(None, 14612, 160)
(None, 14610, 320)
(None, 14610, 320)
(None, 14612, 160)
(None, 29224, 80)
(None, 58448, 40)
(None, 116896, 20)
(None, 116896, 10)


ValueError: Dimensions must be equal, but are 116892 and 116896 for '{{node compile_loss/mse/sub}} = Sub[T=DT_FLOAT](data_1, auto_encoder_32_1/decoder_31_1/conv1d_transpose_141_1/add)' with input shapes: [?,116892,86], [?,116896,10].

In [ ]:
reconstructions = autoencoder.predict(train_data)
train_loss = tf.keras.losses.mae(reconstructions, train_data)

plt.hist(train_loss[None,:], bins=50)
plt.xlabel("Train loss")
plt.ylabel("No of examples")
plt.show()